In [19]:
!which python

/Users/tareksanger/Library/Caches/pypoetry/virtualenvs/lumis-tzzr_5k5-py3.12/bin/python


In [20]:
from lumis.config import config
import os

os.environ["OPENAI_API_KEY"] = config.openai_api_key

In [21]:
import instructor
import openai
from typing import Optional, Literal, List
from pydantic import BaseModel, Field
from enum import Enum


class Thought(BaseModel):
    "The action to be performed."
    thought: str = Field(
        ..., description="You're thought regarding taking the action."
    )
    action: Literal['search', 'lookup', 'finish'] = Field(..., description="The type of action to be performed.")
    assumption: Optional[str] = Field(..., description="Your current assumption. This should not be used for you answer unless validated by accessing the tools at your disposal.")


class SearchAction(BaseModel):
    "The Search query to be preformed."
    entity: str = Field(
        ..., description="The entity to search on Wikipedia.", max_length=15)


class LookupAction(BaseModel):
    "The Lookup query to be preformed."
    keyword: str = Field(
        ..., description="The keyword to lookup in the current passage."
    )


class FinishAction(BaseModel):
    "The Finish query to be preformed."
    answer: str = Field(..., description="The answer to the question.")


client = instructor.from_openai(openai.OpenAI())

In [22]:
messages = [
    {
        "role": "system",
        "content": """
            Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
            (1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
            (2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
            (3) Finish[answer], which returns the answer and finishes the task.

            Choose one of the provided functions to perform the action.
        """,
    },
    {
        "role": "user",
        "content": "Can you list the top 5 resources that Stantec has that might assist in further innovation?",
    },
]

In [23]:
result = client.chat.completions.create(
  model="gpt-4o",
  response_model=Thought,
  messages=messages
)

print(result)

thought='First, I need to gather detailed information about Stantec, particularly regarding their resources for innovation.' action='search' assumption='By researching Stantec, I can find information about their innovation resources.'


In [24]:
import wikipedia

messages.append(
    {
        "role": "assistant",
        "content": f"""
            Action: {result.action}
            Thought: {result.thought}
            Assumption: {result.assumption}
  """,
    }
)

if result.action == 'search':
    search_entity = client.chat.completions.create(
        model="gpt-4-turbo",
        response_model=SearchAction,
        messages=messages
    )
    
    print(search_entity)
    
    page = wikipedia.page(search_entity.entity)
    messages.append(
        {
            "role": "system",
            "content": page.content
        }
    )
    print(page.content)

entity='Stantec'
Stantec Inc. is an international professional services company in the design and consulting industry. The company was founded in 1954, as D. R. Stanley Associates in Edmonton, Alberta, Canada. Stantec provides professional consulting services in planning, engineering, architecture, interior design, landscape architecture, surveying, environmental sciences, project management, and project economics for infrastructure and facilities projects. The company provides services on projects around the world, with over 30,000 employees operating out of more than 450 locations in North America and across offices on six continents.


== History ==
Don Stanley was the first Canadian to earn a Ph.D. in environmental engineering. Attending Harvard University on a Rockefeller Foundation scholarship, he earned his doctorate in 1952 and two years later founded D.R. Stanley & Associates, working as the sole proprietor out of a 250-square-foot (23 m2) office in Edmonton, Alberta. In 1955,

In [25]:
thought = client.chat.completions.create(
    model="gpt-4-turbo",
    response_model=Thought,
    messages=messages
)

print(thought)

thought='From the provided information, I will list the top 5 resources Stantec has that might assist in further innovation.' action='finish' assumption='Stantec has several resources for innovation which can be derived from the their diverse services, global presence, acquisition strategy, major projects, and history of growth and adaptation.'


In [63]:
import pprint

answer = FinishAction(answer="")
if thought.action == 'finish':
  answer = client.chat.completions.create(
    model="gpt-4-turbo",
    response_model=FinishAction,
    messages=messages
  )

In [64]:
pprint.pprint(answer.answer)

("Stantec's top 5 resources that might assist in further innovation include:\n"
 '1. Professional consulting services across various disciplines such as '
 'engineering, architecture, and environmental sciences.\n'
 '2. Global presence with operations in over 450 locations across six '
 'continents, facilitating diverse insights and capabilities.\n'
 '3. Strategic acquisitions strategy, having acquired over 130 firms since '
 '1994, which brings new expertise and technologies into the company.\n'
 '4. Participation in major international projects like the Panama Canal '
 'Expansion and the Keystone Pipeline, which provide vast experiences and '
 'learning opportunities.\n'
 '5. Investment in research and development as evidenced by initiatives such '
 'as their environmental management system and continuous adaptation to market '
 'changes.')


In [17]:
class Thought(BaseModel):
    thought: str = Field(..., description="You're thought regarding the users question.")
    answer: str = Field(..., description="The answer to the users question.")

In [18]:
response = client.chat.completions.create(
    response_model=Thought,
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": """
                Identify opportunities that leverage Stantec’s specific competitive advantages to compete in a market or 
                industry that they don’t currently compete in and explain why they are uniquely positioned to do so.

                Provide your answer in the form of a structured summary. Include a summary of the competitive advantage, 
                financial strengths and weaknesses, and the market or industry that they should enter.
            """,
        }
    ],
)

In [19]:
from IPython.display import display, Markdown

display(Markdown(f"<b>{response.answer}</b>"))

<b>Stantec could explore opportunities in the renewable energy sector, particularly in wind or solar energy projects. Their competitive advantages include expertise in sustainable design and innovative technology solutions, which are crucial in this industry. Financially, Stantec has a stable cash flow and robust profitability, although they face some challenges related to economic cycles. Their experience in large-scale infrastructure projects and a focus on sustainability positions them well for entering this market. Additionally, their international presence could be advantageous in tackling global projects, supporting their expansion into renewable energies.</b>